In [1]:
import torch
torch.cuda.empty_cache()

/home/kudrjavtseviv/DL-framework/.venv-hwr-train-2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import cv2

im1 = cv2.imread()

In [36]:
from losses import DiceLoss

loss_fn = DiceLoss()

In [60]:
import torch.nn as nn
import torch.nn.functional as F

#PyTorch
ALPHA = 100
GAMMA = 10

class FocalLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(FocalLoss, self).__init__()

    def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #first compute binary cross-entropy 
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        BCE_EXP = torch.exp(-BCE)
        focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
                       
        return focal_loss

loss_fn = FocalLoss()

In [84]:
import torch.nn as nn
import torch.nn.functional as F

#PyTorch
ALPHA = 0.15
BETA = 0.85

class TverskyLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(TverskyLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        # inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #True Positives, False Positives & False Negatives
        TP = (inputs * targets).sum()    
        FP = ((1-targets) * inputs).sum()
        FN = (targets * (1-inputs)).sum()
       
        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
        
        return 1 - Tversky
    
loss_fn = TverskyLoss()

In [89]:
a = torch.zeros(10, 10, 2, 2)
a[4][4] = 1

b = torch.zeros(10, 10, 2, 2)
b[6][3] = 1
b[4][4] = 1

c = torch.zeros(10, 10, 2, 2)

# loss_fn(b, a) < loss_fn(a, b)

loss_fn(c, a), loss_fn(c, b), loss_fn(a, b), loss_fn(a, c), loss_fn(b, c), loss_fn(b, b)

(tensor(0.7727),
 tensor(0.8718),
 tensor(0.4048),
 tensor(0.3750),
 tensor(0.5455),
 tensor(0.))

In [92]:
b[6][3]

tensor([[1., 1.],
        [1., 1.]])

In [2]:
import os
cuda_devices = '4, 5'
os.environ['CUDA_VISIBLE_DEVICES'] = cuda_devices

ds_path = '/home/kudrjavtseviv/data/dataset_1'
# ds_path = '/Users/vankudr/Documents/НИР-data/dataset_1'
model_data_path = '/home/kudrjavtseviv/DL-framework/.data'

checkpoints_path = os.path.join(model_data_path, 'checkpoints')
logs_path = os.path.join(model_data_path, 'logs')

In [5]:
from datasets_m import HwrOnPrintedDataModule

# DATA MODULE
dm = HwrOnPrintedDataModule(
    path={
        'train': os.path.join(ds_path, 'train/result'),
        'val': os.path.join(ds_path, 'val/result'),
        'test': os.path.join(ds_path, 'test/result')
    },
    batch_size=1,
    num_workers=0,
    pin_memory=False)

In [8]:
dm.setup(stage='')
loader = dm.train_dataloader()

Compose([
  Resize(always_apply=False, p=1, height=640, width=420, interpolation=1),
  Rotate(always_apply=False, p=1.0, limit=(-35, 35), interpolation=1, border_mode=4, value=None, mask_value=None, rotate_method='largest_box', crop_border=False),
  HorizontalFlip(always_apply=False, p=0.2),
  VerticalFlip(always_apply=False, p=0.2),
  Normalize(always_apply=False, p=1.0, mean=[0.0], std=[1.0], max_pixel_value=255.0),
  ToTensorV2(always_apply=True, p=1.0, transpose_mask=False),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})
Compose([
  Resize(always_apply=False, p=1, height=640, width=420, interpolation=1),
  Normalize(always_apply=False, p=1.0, mean=[0.0], std=[1.0], max_pixel_value=1.0),
  ToTensorV2(always_apply=True, p=1.0, transpose_mask=False),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})
Compose([
  Resize(always_apply=False, p=1, height=640, width=420, interpolation=1),
  Normalize(always_apply=False, p=1.0, mean=[0.0], std

In [29]:
loader.__iter__().__next__()[1][0][0].max()

tensor(0.)

In [3]:
def f_score(pr, gt, beta=1, eps=1e-7, threshold=None, activation='sigmoid'):
    """
    Args:
        pr (torch.Tensor): A list of predicted elements
        gt (torch.Tensor):  A list of elements that are to be predicted
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: IoU (Jaccard) score
    """

    if activation is None or activation == "none":
        activation_fn = lambda x: x
    elif activation == "sigmoid":
        activation_fn = torch.nn.Sigmoid()
    elif activation == "softmax2d":
        activation_fn = torch.nn.Softmax2d()
    else:
        raise NotImplementedError(
            "Activation implemented for sigmoid and softmax2d"
        )

    pr = activation_fn(pr)

    if threshold is not None:
        pr = (pr > threshold).float()


    tp = torch.sum(gt * pr)
    fp = torch.sum(pr) - tp
    fn = torch.sum(gt) - tp

    score = ((1 + beta ** 2) * tp + eps) \
            / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + eps)

    return score


class DiceLoss(nn.Module):
    __name__ = 'dice_loss'

    def __init__(self, eps=1e-7, activation='sigmoid'):
        super().__init__()
        self.activation = activation
        self.eps = eps

    def forward(self, y_pr, y_gt):
        return 1 - f_score(y_pr, y_gt, beta=1., 
                           eps=self.eps, threshold=None, 
                           activation=self.activation)


class BCEDiceLoss(DiceLoss):
    __name__ = 'bce_dice_loss'

    def __init__(self, eps=1e-7, activation='sigmoid', lambda_dice=1.0, lambda_bce=1.0):
        super().__init__(eps, activation)
        if activation == None:
            self.bce = nn.BCELoss(reduction='mean')
        else:
            self.bce = nn.BCEWithLogitsLoss(reduction='mean')
        self.lambda_dice=lambda_dice
        self.lambda_bce=lambda_bce

    def forward(self, y_pr, y_gt):
        dice = super().forward(y_pr, y_gt)
        bce = self.bce(y_pr, y_gt)
        return (self.lambda_dice*dice) + (self.lambda_bce* bce)

In [19]:
from callbacks import SaveImagesCallback, LoadImageToTensorBoard

# MODEL
model = Unet(
    features=[64, 128, 256])
model.configure_optimizers(lr=1e3)

# DATA MODULE
dm = HwrOnPrintedDataModule(
    path={
        'train': os.path.join(ds_path, 'train/result'),
        'val': os.path.join(ds_path, 'val/result'),
        'test': os.path.join(ds_path, 'test/result')
    },
    batch_size=128,
    num_workers=10,
    pin_memory=True if torch.cuda.is_available() else False)

# UTILS
tb_logger = pl.loggers.TensorBoardLogger(save_dir=logs_path)
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor='loss/val',
    dirpath=checkpoints_path, #os.path.join(tb_logger.log_dir, 'checkpoints'), #'checkpoints',
    filename='unet-epoch{epoch:02d}-val_loss{loss/val:.2f}',
    auto_insert_metric_name=False,
    save_top_k=10
)
save_images_callback = SaveImagesCallback(
    image_dir=os.path.join(tb_logger.log_dir, 'images'),
    max_number=3,
    pad=(0, 2)
)
load_images_to_tb = LoadImageToTensorBoard(
    max_number=3,
    pad=(0, 2)
    )

# TRAINER
trainer = pl.Trainer(
    gpus=torch.cuda.device_count(),
    max_epochs=5, 
    log_every_n_steps=2, 
    callbacks=[checkpoint_callback, save_images_callback, load_images_to_tb], 
    logger=tb_logger)
trainer.fit(model, dm)

MisconfigurationException: `Trainer(strategy='ddp_spawn')` or `Trainer(accelerator='ddp_spawn')` is not compatible with an interactive environment. Run your code as a script, or choose one of the compatible strategies: Trainer(strategy=None|dp|tpu_spawn). In case you are spawning processes yourself, make sure to include the Trainer creation inside the worker function.

: 